<a href="https://colab.research.google.com/github/KIMCAT33/ML-Pipeline/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from __future__ import absolute_import, division, print_function, unicode_literals
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras import models
from keras import layers
from google.cloud import bigquery
from google.colab import auth

auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
project_id = 'pipeliner'

client = bigquery.Client(project=project_id)

In [31]:
query = """
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks
FROM
    publicdata.samples.natality
WHERE year > 2000
"""

df = client.query(query + "LIMIT 10000").to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.131954,True,22,1,41.0
1,7.980734,False,28,1,44.0
2,8.675190,False,33,1,41.0
3,5.346210,True,25,1,35.0
4,6.999677,False,29,1,39.0


In [32]:
columns = ['is_male', 'mother_age', 'plurality', 'gestation_weeks']
df.dropna(how="any", inplace=True)

x=df[columns]
y = df['weight_pounds']

x.loc[x["is_male"] == True, "is_male"] = 1
x.loc[x["is_male"] == False, "is_male"] = 0


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)

In [0]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_x.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [35]:
k = 4
num_val_samples = len(train_x) // k
num_epochs = 20
all_mae_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    # 검증 데이터 준비: k번째 분할
    val_data = train_x[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_y[i * num_val_samples: (i + 1) * num_val_samples]

    # 훈련 데이터 준비: 다른 분할 전체
    partial_train_data = np.concatenate(
        [train_x[:i * num_val_samples],
         train_x[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_y[:i * num_val_samples],
         train_y[(i + 1) * num_val_samples:]],
        axis=0)

    # 케라스 모델 구성(컴파일 포함)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

처리중인 폴드 # 0
Train on 5921 samples, validate on 1973 samples
Epoch 1/20
5921/5921 [==============================] - 24s 4ms/step - loss: 1.4793 - mean_absolute_error: 0.9439 - val_loss: 1.2450 - val_mean_absolute_error: 0.8754
Epoch 2/20
5921/5921 [==============================] - 23s 4ms/step - loss: 1.3300 - mean_absolute_error: 0.9040 - val_loss: 1.7217 - val_mean_absolute_error: 1.0522
Epoch 3/20
5921/5921 [==============================] - 24s 4ms/step - loss: 1.2905 - mean_absolute_error: 0.8863 - val_loss: 1.3409 - val_mean_absolute_error: 0.9107
Epoch 4/20
5921/5921 [==============================] - 23s 4ms/step - loss: 1.2742 - mean_absolute_error: 0.8824 - val_loss: 1.3247 - val_mean_absolute_error: 0.9105
Epoch 5/20
5921/5921 [==============================] - 23s 4ms/step - loss: 1.2572 - mean_absolute_error: 0.8769 - val_loss: 1.8037 - val_mean_absolute_error: 1.0795
Epoch 6/20
5921/5921 [==============================] - 23s 4ms/step - loss: 1.2575 - mean_absolute_error

In [0]:
model.save_weights('./natality.h5', overwrite=True)